# Magical snakes

<center>
<figure style="display: block;
  margin-left: auto;
  margin-right: auto;
  width: 50%;
}">
<img
    src="images/DALL·E 2023-03-18 20.01.53 - a cartoon snake with a black hoodie typing on a laptop (cleaned).png"
    alt="a cartoon snake with a black hoodie typing on a laptop"
    width="768px"
/>
<footer><small>Created by DALL·E 2 and adjusted by me</small></footer>
</figure>
</center>

## Everything is an object
Including data, classes, types, and code

which are referenced / "pointed to" by variables (names)


```python
a = b = 2

print(a, b)

a = "Bouvet"

print(a)
print(b)
```


## Specimens

<center>
    <figure>
        <img
             src="https://d38mjcf1nvhlba.cloudfront.net/media/o/1656679102/dsc-3326-1.jpg"
             alt="Two penguins at the Aquarium in Bergen. From left to right: Ping Kong, Ealinor Roosivelt"
             style="height: 768px;"
        />
        <footer><small>Two penguins at the Aquarium in Bergen. From left to right: Ping Kong, Ealinor Roosivelt
            <br>&copy; The Aquarium in Bergen</small></footer>
    </figure>
</center>

Much of Python's syntax have methods / functions associated with them.
In some circumstances, using them can make the code _much_ easier to read and understand.

These, and some other start, and end with `__` (called "dunder").

In [ ]:
def __init__(self, name: str):
    self._name = name

`type` is a function which may give the type of the object, or create a new type depending on how it is called (1 vs 3 arguments)

`__new__` is a static method every object have (because they inherit from `object`)

There is nothing stopping us from changing the name of the method itself, as long as it has the key `"__init__"` in the `dict` we provide `type`.

In [ ]:
Penguin = type("Penguin", (object,), {
    "__init__": __init__,
    "name": property(lambda self: self._name)
})

penguin = Penguin.__new__(Penguin)
Penguin.__init__(penguin, "Pingrid Alexandra")

In [ ]:
class Penguin:
    def __init__(self, name: str):
        self._name = name

    @property
    def name(self):
        return self._name
        
penguin = Penguin("Pingrid Alexandra")

print(penguin.name)

## 🪄 Time for a little deeper (everyday) magic


<center>
    <figure>
        <img
             src="https://images.unsplash.com/photo-1614767629805-3bbcf6e26c7d?ixlib=rb-4.0.3&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=1760&q=80"
             alt="Photograph of various dice with different number of sides laying on a tabletop board"
             style="height: 768px;"
        />
        <footer><small>Photo by <a href="https://unsplash.com/@nika_benedictova?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText">Nika Benedictova</a> on <a href="https://unsplash.com/s/photos/dice?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText">Unsplash</a></small></footer>
    </figure>
</center>
    

## 🪄 Time for a little deeper (everyday) magic

Many tabletop role playing games uses many different dice. Typically, they have 4, 6, 12, 12, and 20 sides (one of these are not like the others)

Let's make one!

A simple die with `n` sides

```python
import random
import math
from typing import Self


class Die:
    def __init__(self, sides: int):
        self.sides = sides

    def roll(self) -> int:
        # An alternative way of getting a fair roll
        # math.floor(random.random() * self.sides) + 1
        return random.randrange(1, self.sides + 1)

    def __str__(self):
        # A common dotation for a die with `n` sides is d<n>
        return f"d{self.sides}"
    
    def __repr__(self):
        # generally, __repr__ should give a string representation which, if interpreterd
        # should create the relevant object (or a copy of it)
        # However, in this case, it is more convinient to use the simple represntation now
        # If we where to do it properly though, we could write
        # return f"{self.__class__.__name__}({self.sides})"
        return str(self)
    
    # What about comparison?
    # Do we care about care if they are distinct objects, or is it
    # sufficient that they have the same number of sides?
    def __eq__(self, other: Self):
        if not isinstance(other, self.__class__):
            return False
        return self.sides == other.sides
    
    def __hash__(self):
        # Should not be here if __eq__ is not defined
        return hash(str(self))
```

## 🪄 Time for a little deeper (everyday) magic

A single die is well and good, but usually we may need more than one to roll simultaneously
For instance when we roll for damage on another creature or monster
It can be _very_ tedious to roll a single die over and over, and then summarize all the numbers

Let's do something about that!
After all, that is one of the primary reason humanity invented the computer

Multiple dice gathered together

The reason we use inheritance is laziness;
I don't have to rewrite mush of the code, and you don't have to watch me doing it.
There is also just so much vertical space on these slides


```python
class Dice(Die):
    def __init__(self, sides: int, num: int = 1):
        super().__init__(sides)
        self.num = num

    def __rmul__(self, other: int):
        if not isinstance(other, int):
            # NOTE: It RETURNS a value, and does not throw an exception
            # ref. https://docs.python.org/3/library/constants.html#NotImplemented
            return NotImplemented
        return self.__class__(self.sides, other)

    def roll(self):
        total = 0
        for _ in range(self.num):
            total += super().roll()
        return total
    
    def __str__(self):
        return f"{self.num if self.num > 1 else ''}{super().__str__()}"

```

examples

```python
d10 = Dice(10)
_6d10 = 6 * d10
_6d10.roll()    
```

## 🪄 Time for a little deeper (everyday) magic

Sometimes, we have a (constant) modifier.
It can be positive, or negative

Dice with modifiers


```python
class Dice(Dice):
    def __init__(self, sides: int, num: int = 1, modifier: int = 0):
        super().__init__(sides, num)
        self.modifier = modifier
    
    def __add__(self, other):
        return self.__class__(self.sides, self.num, other)
    
    def __str__(self):
        modifier = ""
        if self.modifier > 0:
            modifier = f" + {self.modifier}"
        elif self.modifier < 0:
            modifier = f" - {abs(modifier)}"
        return super().__str__() + modifier
    
    def roll(self):
        return super().roll() + self.modifier

    # Using `.roll()` is a bit tedious.
    # How about writing `()` instead?
    def __call__(self):
        return self.roll()
```

Examples

```python

die = 2 * Dice(12) + 2
```

## 🧙‍♂️Time for (real) magic

Wouldn't it be nice to write

```python
Dice._4d12
```

and have a finished result written to the console?

## 🧙‍♂️Time for (real) magic

In [ ]:
import re

In [ ]:
_regex = re.compile(r"^(?P<number>\d+)?d(?P<sides>\d+) *((?P<sign>[-+]) *(?P<modifier>\d+)?)?$")
_Dice = Dice  # We will redefine `Dice` later (again), but need a reference to it here

In [ ]:
def parse(text: str) -> Dice:
    match = _regex.match(text)
    if match is None:
        raise ValueError("Invalid format")
    match = match.groupdict()
    sides = int(match["sides"])
    number = int(match.get("number", 1))
    modifier = int(match.get("modifier", 0))
    if match["sign"] == "-":
        modifier = -modifier
    return _Dice(sides, number, modifier)

Content warning (here be regualr expressions)

Here is a small regualr expressions. It is not very magical (at least when you're the one writing it, and you remember what it does)

```python
class Dice:
    def __getattr__(self, item: str):
        item = (
            item
            .removeprefix("_")
            .replace("_", " ")
            .replace("p", "+")
            .replace("m", "-")
        )
        return parse(item)
```

Unfortunately, we have to instantiate the object before it behaves as expected.
There is a solution for such problems.
Since `__getattr__` is defined on the class, but used on instances, we simply need to do the equivalent on the class itself.

```python
class Meta(type):
    def __getattr__(cls, item: str):
        item = (
            item
            .removeprefix("_")
            .replace("_", " ")
            .replace("p", "+")
            .replace("m", "-")
        )
        return parse(item)

class Dice(metaclass=Meta):
    pass
```

Using the `@staticmethod`, or `@classmethod` is tempting, but unfortunately does not do what we want;
the arbitrary attribute is still not defined on the class

```python
class Dice:
    @staticmethod
    def __getattr__(item: str):
        item = (
            item
            .removeprefix("_")
            .replace("_", " ")
            .replace("p", "+")
            .replace("m", "-")
        )
        return parse(item)
```
